# JON'S HiML Competition RFG_Log Model (v 2.6)
## Make Date: 04/13/18
I found out RFG has tuning parameters. Let's see if we can improve prediction errors of each combo done in v 2.2

In [2]:
#Some initialization procedures:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import random
import time
import math
# load in data files
# FILE_DIR = '../input/hawaiiml-data'
# for f in os.listdir(FILE_DIR):
#     print('{0:<30}{1:0.2f}MB'.format(f, 1e-6*os.path.getsize(f'{FILE_DIR}/{f}')))
FILE_DIR = '../Sprint09alt_Machine_Learning_Hawaii_Kaggle_Competition'
df_train = pd.read_csv(f'{FILE_DIR}/train.csv', encoding='ISO-8859-1') #write training data to dataframe
df_test = pd.read_csv(f'{FILE_DIR}/test.csv', encoding='ISO-8859-1') # Read the test data

#define the error function:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_true) - np.log1p(y_pred))**2))

# Prediction Target
We want to predict the quantity data field.    
By convention, we define this target as 'y'.  
We know quantity is highly skewed. See if a log transform will help.  

In [3]:
y = df_train.quantity
logy = np.log1p(df_train.quantity)

# Define ML Predictors
build a function to go through each combination of predictors

### Here is the list of columns we can choose predictors from. To keep it simple, just select from numeric data types.

In [4]:
print('Column Names & Data Types: \n', df_train.dtypes)

Column Names & Data Types: 
 id               int64
date            object
time            object
invoice_id       int64
stock_id         int64
customer_id      int64
country         object
description     object
unit_price     float64
quantity         int64
dtype: object


## Define feature combination picker

In [5]:
ls_AllPredictors = ['invoice_id', 'stock_id', 'customer_id', 'unit_price']

# https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
#build combos:
ls_PredictorCombos = [list(combo[1]) for combo in enumerate(powerset(ls_AllPredictors), 1)]
print (ls_PredictorCombos[2])
# display(df_train[ls_PredictorCombos[2]])
def GetX(comboID,df):
    return df[ls_PredictorCombos[comboID]]
# display(GetX(2))

['stock_id']


# Score Different Random Forest Regressor Models
Iteratively try different parameter tunings on each predictor combo

In [6]:
def Run_RFG(SeedVal, X, logy, i_n_estimators=10,i_min_samples_leaf=1, i_max_leaf_nodes=None, pct_max_features = 100, i_n_jobs=1):
    #fit, predict, and then evaluate the passed in mod = el using passed in values (e.g. training set)
    #return back RMSLE value and trained model
    from sklearn.ensemble import RandomForestRegressor

    myModel = RandomForestRegressor(
        n_estimators = i_n_estimators, 
        min_samples_leaf = i_min_samples_leaf,
        max_leaf_nodes = i_max_leaf_nodes, 
        max_features = pct_max_features / 100, 
        oob_score = True,
        n_jobs=i_n_jobs, 
        random_state=SeedVal)
    train_X, val_X, train_y, val_y = train_test_split(X, logy,random_state = SeedVal) #split training data into a test and train part
    myModel.fit(train_X, train_y)
    predicted_vals = np.expm1(myModel.predict(val_X)) #transform predicted values from log to "normal" Y value
    return rmsle(np.expm1(val_y), predicted_vals), myModel #include transform of val_Y values from log to "normal" Y value

In [7]:
def ScoreCombos (SeedVal, df_X, logy, ScoreAllCombos, ls_PredictorCombos, i_n_estimators=10,i_min_samples_leaf=1, i_max_leaf_nodes=None, pct_max_features = 100, i_n_jobs=1):
    #find RMSLE error of each predictor combination
    #return dataframe of results: ComboID	err	Preds
    MinErr = 100000000
#     df_Track = pd.DataFrame()
    bestModel = None
    if ScoreAllCombos:
        for comboID in range(1,len(ls_PredictorCombos)):
            X = GetX(comboID, df_X)
            TrainErr,myModel = Run_RFG(SeedVal, X, logy, i_n_estimators,i_min_samples_leaf, i_max_leaf_nodes, pct_max_features, i_n_jobs)
    #         df2 = pd.DataFrame([[comboID, TrainErr, ','.join(ls_PredictorCombos[comboID])]],columns=['ComboID','err','Preds'])
    #         if df_Track.shape[0] >0:
    #             df_Track = pd.concat([df2, df_Track])
    #         else:
    #             df_Track = df2.copy(deep = True)
            if TrainErr < MinErr:
                MinErr = TrainErr
                bestComboID = comboID
                bestModel = myModel
                print ('Best Combo: ', bestComboID, ' Params: ', ls_PredictorCombos[comboID], ' Err: ', TrainErr)
    else:
            X = df_X[ls_PredictorCombos]
            TrainErr,myModel = Run_RFG(SeedVal, X, logy, i_n_estimators,i_min_samples_leaf, i_max_leaf_nodes, pct_max_features, i_n_jobs)
    #         df2 = pd.DataFrame([[comboID, TrainErr, ','.join(ls_PredictorCombos[comboID])]],columns=['ComboID','err','Preds'])
    #         if df_Track.shape[0] >0:
    #             df_Track = pd.concat([df2, df_Track])
    #         else:
    #             df_Track = df2.copy(deep = True)
            if TrainErr < MinErr:
                MinErr = TrainErr
                bestComboID = 0
                bestModel = myModel
                print ('Best Combo: ', bestComboID, ' Params: ', ls_PredictorCombos, ' Err: ', TrainErr)
        
    print ('fin')
#     return df_Track
    return MinErr, bestComboID, bestModel

In [17]:
ls_PredictorCombos = ['invoice_id', 'customer_id', 'unit_price']
SeedVal = 0
curErr, curComboID, curModel = ScoreCombos(SeedVal, df_train, logy, True, ls_PredictorCombos)

ValueError: Expected 2D array, got 1D array instead:
array=[   7. 1827. 1248. ...   74. 2376. 2143.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [9]:
def leftORRight():
    ls_leftORright = [-1,1]
    return ls_leftORright[random.randint(0,1)]
#     return -1 #for now, force left

def HopToVal(HopFactor, CurVal, r_vals):
    #hop to another value within range R_vals that is at most HopFactor% away from current value
    #returned value is float so you'll need to round and cast (Say to int) if needed
    xrMin = r_vals[0]
    xrMax = r_vals[len(r_vals)-1]
    LR = leftORRight()
    if LR < 0:
        HopDistMax = xrMin-CurVal
    else:
        HopDistMax = xrMax-CurVal
    x1 = CurVal + HopDistMax * random.uniform(0,HopFactor)
    if x1 < xrMin: x1=xrMin
    return x1

def TuneParams_RW(df_X, logy, ScoreAllCombos, ls_PredictorCombos, MaxWalks,  r_n_estimators, r_min_samples_leaf, r_max_leaf_nodes, r_pct_max_features, i_n_jobs, SeedVal):
    # find best model param combo using random walk
    random.seed(SeedVal)
    bestModel = None
    bestComboID = 0
    bestErr = 100000000
    #set some intial values for each tuning parameter (they won't be used - just need to give values to do initial hop on)
    n_estimators = r_n_estimators[0]
    min_samples_leaf = r_min_samples_leaf[0]
    max_leaf_nodes = r_max_leaf_nodes[0]
    pct_max_features = r_pct_max_features[0]
    for Walki in range(0,MaxWalks):
        #assign values:
        print ('Evaluating tuning params: n_estimators: ', n_estimators, 'min_samples_leaf: ', min_samples_leaf, 'max_leaf_nodes: ', max_leaf_nodes,)
        n_estimators = int(round(HopToVal(1,n_estimators,r_n_estimators),0))
        min_samples_leaf = int(round(HopToVal(1,min_samples_leaf, r_min_samples_leaf),0))
        max_leaf_nodes = int(round(HopToVal(1,max_leaf_nodes, r_max_leaf_nodes),0))
        pct_max_features = int(round(HopToVal(1,pct_max_features, r_pct_max_features),0))
        curErr, curComboID, curModel = ScoreCombos(SeedVal, df_X, logy, ScoreAllCombos, ls_PredictorCombos, n_estimators, min_samples_leaf, max_leaf_nodes, pct_max_features, i_n_jobs)
        if curErr < bestErr:
            bestErr = curErr
            bestComboID = curComboID
            bestModel = curModel
            print ('!!!New best tuning: \npredictors: ', curModel.get_params(deep=True),  ' model error: ', bestErr, ' n_estimators: ', n_estimators, 'min_samples_leaf: ', min_samples_leaf, 'max_leaf_nodes: ', max_leaf_nodes,)
        # clear_output()
    return bestErr, bestComboID, bestModel
#     print ('--- Completed sim runs for tgtRR: ',tgtRR)
#     print ('--- %s execution time in seconds ---' % (time.time() - start_time))
#     return [ls_best, pd_hist]

# TuneParams_RW(df_train, logy, False, ['invoice_id', 'customer_id', 'unit_price'], 1, range(10,100), range(1,2), range(5,50), 4, 0)



In [10]:
# TMax=1.0
# TMin = 0.01
# Talpha = 0.75
# MaxWalks = 500
# Tstepcnt = 0 #initialize counter of number of steps between 1st T and T_Min
# #calculate number of walks per Tstep
# T = TMax #setup T for calculation
# while T >= TMin:
#     T = T * Talpha
#     Tstepcnt += 1

# WalksPerTstep = int (MaxWalks / Tstepcnt) #walks per Tstep
# print ('Walks per T step: ', WalksPerTstep)

In [13]:
def prAccept(prevErr, curErr, T):
    try:
        prob = math.exp((prevErr - curErr)/T)
    except OverflowError:
        prob = 1 #calculated number is infinite. make prob = 1
    return prob

def TuneParams_SA(df_X, logy, ScoreAllCombos, ls_PredictorCombos, MaxWalks, NoPrgNextT, TMin, Talpha,
                  r_n_estimators, r_min_samples_leaf, r_max_leaf_nodes, r_pct_max_features, i_n_jobs, SeedVal):
    # find best model param combo using simulated annealing
    random.seed(SeedVal)
    bestModel = None
    bestComboID = 0
    bestErr = 100000000
    
    #we need 2 error measurements (prevErr and curErr) before entering SA. do this by doing 2 random walk results
    prevErr, prevComboID, prevModel = TuneParams_RW(df_X, logy, ScoreAllCombos, ls_PredictorCombos, 1, 
                r_n_estimators, r_min_samples_leaf, r_max_leaf_nodes, r_pct_max_features, i_n_jobs, SeedVal)
    if prevErr < bestErr:
        bestErr = prevErr
        bestComboID = prevComboID
        bestModel = prevModel
        print ('!!!New best tuning: \npredictors: ', bestModel.get_params(deep=True),  ' model error: ', bestErr, ' n_estimators: ', bestModel.n_estimators, 'min_samples_leaf: ', bestModel.min_samples_leaf, 'max_leaf_nodes: ', bestModel.max_leaf_nodes,)
    curErr, curComboID, curModel = TuneParams_RW(df_X, logy, ScoreAllCombos, ls_PredictorCombos, 1, 
                 r_n_estimators, r_min_samples_leaf, r_max_leaf_nodes, r_pct_max_features, i_n_jobs, SeedVal)
    if curErr < bestErr:
        bestErr = curErr
        bestComboID = curComboID
        bestModel = curModel
        print ('!!!New best tuning: \npredictors: ', bestModel.get_params(deep=True),  ' model error: ', bestErr, ' n_estimators: ', bestModel.n_estimators, 'min_samples_leaf: ', bestModel.min_samples_leaf, 'max_leaf_nodes: ', bestModel.max_leaf_nodes,)    
    WalkNo = 2
    
    #get ready to enter SA loop:
    TMax=1.0
    Tstepcnt = 0 #initialize counter of number of steps between 1st T and T_Min
    #calculate number of walks per Tstep
    T = TMax #setup T for calculation
    while T >= TMin:
        T = T * Talpha
        Tstepcnt += 1
    WalksPerTstep = int (MaxWalks / Tstepcnt) #walks per Tstep
    print ('Walks per T step: ', WalksPerTstep)
    T = TMax #Reset T
    NoPrgWalkCnt = 0
    #enter SA loop:
    while T > TMin:
        for iterAtT in range (0,WalksPerTstep): #for each temp, do sim. runs
            #advance iterators:
            WalkNo = WalkNo + 1
            if WalkNo > MaxWalks:
                break
            if iterAtT>0:
                prAcceptVal = prAccept(prevErr, curErr, T) #PROBABILITY OF ACCEPTING SOLUTION. PR > 1 FOR IMPROVED SOLUTION. PR [0,1] FOR WORSTEND SOLUTION
                prRandVal = random.uniform(0,1.0)
                # print ('Acceptance Consideration: T: ', T, ' prAccept: ', prAcceptVal, ' prRandVal: ', prRandVal)
                if prAcceptVal > prRandVal: #accept current solution & base next solution on current solution
                    n_estimators = int(round(HopToVal(T,curModel.n_estimators,r_n_estimators),0))
                    min_samples_leaf = int(round(HopToVal(T,curModel.min_samples_leaf, r_min_samples_leaf),0))
                    max_leaf_nodes = int(round(HopToVal(T,curModel.max_leaf_nodes, r_max_leaf_nodes),0))
                    pct_max_features = int(round(HopToVal(T,curModel.max_features*100, r_pct_max_features),0))
                    prevModel = curModel
                    prevComboID = curComboID
                    prevErr = curErr
                else: #reject current solution & base next solution on prior solution
                    n_estimators = int(round(HopToVal(T,prevModel.n_estimators,r_n_estimators),0))
                    min_samples_leaf = int(round(HopToVal(T,prevModel.min_samples_leaf, r_min_samples_leaf),0))
                    max_leaf_nodes = int(round(HopToVal(T,prevModel.max_leaf_nodes, r_max_leaf_nodes),0))
                    pct_max_features = int(round(HopToVal(T,prevModel.max_features*100, r_pct_max_features),0))                    
            else: #reset to best position at each new temp
                    print ('Resetting to best params')
                    n_estimators = int(round(HopToVal(T,bestModel.n_estimators,r_n_estimators),0))
                    min_samples_leaf = int(round(HopToVal(T,bestModel.min_samples_leaf, r_min_samples_leaf),0))
                    max_leaf_nodes = int(round(HopToVal(T,bestModel.max_leaf_nodes, r_max_leaf_nodes),0))
                    pct_max_features = int(round(HopToVal(T,bestModel.max_features*100, r_pct_max_features),0))
                    prevModel = bestModel
                    prevComboID = bestComboID
                    prevErr = bestErr                    
            #assign values:
            print ('\nWalkNo: ', WalkNo)
            print ('Evaluating tuning params: n_estimators: ', n_estimators, 'min_samples_leaf: ', min_samples_leaf, 'max_leaf_nodes: ', max_leaf_nodes, ' pct_max_features: ', pct_max_features)
            curErr, curComboID, curModel = ScoreCombos(SeedVal, df_X, logy, ScoreAllCombos, ls_PredictorCombos, n_estimators, min_samples_leaf, max_leaf_nodes, pct_max_features, i_n_jobs)
            if curErr < bestErr:
                bestErr = curErr
                bestComboID = curComboID
                bestModel = curModel
                print ('!!!New best tuning: \npredictors: ', bestModel.get_params(deep=True),  ' model error: ', bestErr, ' n_estimators: ', bestModel.n_estimators, 'min_samples_leaf: ', bestModel.min_samples_leaf, 'max_leaf_nodes: ', bestModel.max_leaf_nodes,)    
                NoPrgWalkCnt =0 
            else:
                NoPrgWalkCnt += 1
                if NoPrgNextT < NoPrgWalkCnt / MaxWalks: #leave current temperature step if no progress made
                    break
        #advance iterators:
        T = T * Talpha
        NoPrgWalkCnt = 0
        if WalkNo > MaxWalks:
            break
            
    return bestErr, bestComboID, bestModel

TuneParams_SA(df_train, logy, True, ['invoice_id', 'customer_id', 'unit_price'], 50, 0.80, 0.01, 0.75, range(100,5000), range(1,2), range(5,5000), range(1,100), -1, 0)


Evaluating tuning params: n_estimators:  100 min_samples_leaf:  1 max_leaf_nodes:  5


KeyboardInterrupt: 

## OBSERVATION:  customer id & unit price are a part of the lowest 4 scores

In [22]:
df_Track.sort_values(by=['err'])

NameError: name 'df_Track' is not defined

# Submit Model's Predictions
## First, output model's predictions for test data set:

In [12]:
test_X = df_test[ls_mypredictors]
# Use the model to make predictions
predicted_vals = myModel.predict(test_X)
predicted_vals = np.expm1(myModel.predict(test_X)) #transform from log to "normal" Y value
# We will look at the predicted prices to ensure we have something sensible.
print(predicted_vals)

[  4.1609216   15.53076939  13.03261551 ...,   2.44194547   1.01317254
   2.05402861]


## Next, submit predicted values

In [13]:
my_submission = pd.DataFrame({'Id': df_test.id, 'quantity': predicted_vals})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)